# Privacy-Preserving Record Linkage (PPRL): Investigating Dataset Extension Attacks

## Setup

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import string
import sys
from tqdm.notebook import tqdm

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Torchvision version', torchvision.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

System Version: 3.10.9 (main, Nov 22 2023, 17:43:49) [Clang 15.0.0 (clang-1500.0.40.1)]
PyTorch version 2.2.1
Torchvision version 0.17.1
Numpy version 1.26.4
Pandas version 2.2.1


## Define Dataset

In [48]:
class BloomFilterDataset(Dataset):
    def __init__(self, data_dir, isLabeled=False, all_two_grams=None):
        self.isLabeled = isLabeled
        self.allTwoGrams = all_two_grams
        self.data = pd.read_csv(data_dir, sep='\t')  # Load TSV data
        if self.isLabeled:
            # For reidentified data, extract labels (2-grams) from names
            self.data['label'] = self.data.apply(lambda row: self.extract_two_grams(row['surname'], row['firstname']), axis=1)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        bloom_filter = self.data.iloc[idx]['bloomfilter']
        bloom_filter_tensor = self.bloomfilter_to_tensor(bloom_filter)

        if self.isLabeled:
            label = self.data.iloc[idx]['label']
            label_tensor = self.label_to_tensor(label)
            return bloom_filter_tensor, label_tensor
        else:
            # For unlabeled data (not_reidentified_individuals.tsv), just return the Bloom Filter
            return bloom_filter_tensor
    
    def extract_two_grams(self, surname, firstname):
        full_name = f"{surname} {firstname}".replace('"', '').replace('.', '').replace('/', '').strip()
        full_name = full_name.lower()  # Normalize to lowercase for consistency
        return [full_name[i:i+2] for i in range(len(full_name)-1) if ' ' not in full_name[i:i+2]]
    
    def bloomfilter_to_tensor(self, bloom_filter_str):
        bloom_filter_array = np.array([int(bit) for bit in bloom_filter_str], dtype=np.float32)
        return torch.tensor(bloom_filter_array)
    
    def label_to_tensor(self, label_two_grams):
        label_vector = np.zeros(len(self.allTwoGrams), dtype=np.float32)
        
        # Set 1 for the 2-grams present in the name
        for gram in label_two_grams:
            if gram in self.allTwoGrams:
                index = self.allTwoGrams.index(gram)
                label_vector[index] = 1
        
        return torch.tensor(label_vector)

## Create Datasets 

In [60]:
#Generate all 2-grams
alphabet = string.ascii_lowercase
all_two_grams = []
for a in alphabet:
    for b in alphabet:
        all_two_grams.append(a+b)
# Get a dictionary associating each 2-gram with an index
two_gram_dict = {i: two_gram for i, two_gram in enumerate(all_two_grams)}
# Create Datasets
data_labeled = BloomFilterDataset('./data/eves_information/reidentified_individuals.tsv', isLabeled=True, all_two_grams=all_two_grams)
data_not_labeled = BloomFilterDataset('./data/eves_information/not_reidentified_individuals.tsv', isLabeled=False, all_two_grams=all_two_grams)

## Create Dataloader

In [62]:
dataloader_labeled = DataLoader(data_labeled, batch_size=32, shuffle=True)
dataloader_not_labeled = DataLoader(data_not_labeled, batch_size=32, shuffle=False)

## Pytorch Datamodel

In [70]:
class BloomFilterToTwoGramClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, num_two_grams=676):
        super(BloomFilterToTwoGramClassifier, self).__init__()
        
        # Define the layers for multi-label classification of 2-grams
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),  # Input layer mapping Bloom filter bits to hidden layer
            nn.ReLU(),                        # Activation function for non-linearity
            nn.Dropout(0.3),                   # Dropout to prevent overfitting
            nn.Linear(hidden_dim, hidden_dim), # Second hidden layer
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_two_grams), # Output layer for 2-gram predictions
            nn.Sigmoid()                       # Sigmoid for multi-label probabilities
        )

    def forward(self, x):
        # Forward pass through the model
        output = self.model(x)
        return output

## Testing Area

In [69]:
# First entry for reference (labeled data):
# surname                                                  Hegarty
# firstname                                    Miss. Hanora "Nora"
# bloomfilter    0000000010100011000100000101000000000000100000001000000000100111011000001001100100000100000000001000000000000000001000100000000000010010000000000000100010001110110111000000000000100000000100000001010000000000100101000011000010001010000001000000000000000000001000011010011001000100000011100100000000000011000100100000110011000000000010000000000010000000000000110000000110000000000010000000011100000001000000100000001100101011001000000000010000001000000000001000010110110000000001001000100001010111010000000010000000111000000000010010110000000000001000000101010001000000001000001000010000100110000111001110000000001010011110000100000000000100000001100001100000000000010000000000000000000000100000000010000001000000000011100000000000001000101000010100001001000011000000000010001100000000100000001000001000000000100000101000000000000000000010000000100000000100001000000100000000000000011100000001001000000001100010000001000001000000000000010100100000000110101110010000010000010100000000011000001000000001110000101001000010101111
# uid                                                          654
# Name: 0, dtype: object
data_labeled = BloomFilterDataset('./data/eves_information/reidentified_individuals.tsv', isLabeled=True, all_two_grams=all_two_grams)
data_not_labeled = BloomFilterDataset('./data/eves_information/not_reidentified_individuals.tsv', isLabeled=False, all_two_grams=all_two_grams)
#print('Length Labeled data:', len(data_labeled))
#print('Length Unlabeled data:', len(data_not_labeled))

bloomfilter_tensor, label_tensor = data_labeled[0]

#print('Bloom Filter Tensor:', bloomfilter_tensor)
#print('Bloom Filter Tensor Shape:', bloomfilter_tensor.shape)
#print('Label Tensor:', label_tensor)
#print('Label Tensor Shape:', label_tensor.shape)

for bloomfilter_tensors, label_tensors in dataloader_labeled:
    print('Bloom Filter Tensor Shape:', bloomfilter_tensors.shape)
    print('Label Tensor Shape:', label_tensors.shape)
    print(label_tensors)
    break

Bloom Filter Tensor Shape: torch.Size([32, 1024])
Label Tensor Shape: torch.Size([32, 676])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
